In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re
import matplotlib.pyplot as plt

In [2]:
from IPython.display import display

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [4]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

### Load HDF5 file from working directory 

In [5]:
os.chdir(r'C:\Users\Sellit\Desktop\BA_Coding\Data\1final_out')

In [6]:
start4=datetime.now()

df_out = pd.read_hdf('DataFrame_1_df_out.h5', 'df_out')

end4 = datetime.now()
print(end4 - start4)

0:00:02.124550


#### only first 5 pages

In [7]:
df_out = df_out.groupby('session_id').head(5)

In [8]:
df_out_head = df_out.groupby('session_id').head(1)

### Check out dataframe

In [9]:
df_out = df_out.reset_index(drop=True)

In [10]:
num_of_sessions = df_out['session_id'].nunique()

In [11]:
num_of_sessions

9760

In [12]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48800 entries, 0 to 48799
Data columns (total 44 columns):
idx                        48800 non-null int32
purchase                   48800 non-null bool
session_id                 48800 non-null object
num_pages                  48800 non-null int64
created                    48800 non-null datetime64[ns]
lastActivity               48800 non-null datetime64[ns]
startTime                  48800 non-null datetime64[ns]
endTime                    48800 non-null datetime64[ns]
engagementTime             48800 non-null float64
visitTime                  48800 non-null float64
total_engagementTime       48800 non-null float64
duration                   48800 non-null float64
page_cat                   48800 non-null object
scroll                     48800 non-null float64
uri                        48800 non-null object
websitePage                48800 non-null object
ref_gen                    31015 non-null object
page_id                  

### Add features about mouse movement behavior

#### Load DataFrame 2

In [13]:
start4=datetime.now()

df_purch = pd.read_hdf('for_further_purchases_5_pages_df.h5', 'df')

end4 = datetime.now()
print(end4 - start4)

0:00:09.889081


In [14]:
start4=datetime.now()

df_nonpurch = pd.read_hdf('for_further_nonpurchases_5_pages_df.h5', 'df')

end4 = datetime.now()
print(end4 - start4)

0:00:27.314937


##### Merge purchase and Non-Purchase Sessions

In [15]:
df_purch['purchase'] = True
df_nonpurch['purchase'] = False

In [16]:
df = df_purch.append(df_nonpurch, ignore_index=True)

C:\Users\Sellit\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


#### Load DataFrame 3

In [17]:
start4=datetime.now()

df_purch = pd.read_hdf('purchases_5_pages_for_merge_df.h5', 'df')

end4 = datetime.now()
print(end4 - start4)

0:00:00.362267


In [18]:
start4=datetime.now()

df_nonpurch = pd.read_hdf('nonpurchases_5_pages_for_merge_df.h5', 'df')

end4 = datetime.now()
print(end4 - start4)

0:00:01.085086


##### Merge purchase and Non-Purchase Sessions

In [19]:
df_purch['purchase'] = True
df_nonpurch['purchase'] = False

In [20]:
df2 = df_purch.append(df_nonpurch, ignore_index=True)

C:\Users\Sellit\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


### Merge all three DataFrames

In [21]:
# first row of every page of DataFrame 2
df_head = df.groupby(['id', 'startTime']).head(1)

In [22]:
df_out2 = pd.merge(df_out, df_head[['id', 'startTime','mean_velocity', 'dist_on_page', 'time_no_movement']],left_on=['session_id', 'startTime'],right_on=['id', 'startTime'], how='left')

In [23]:
list_col1 = list(df2.columns[pd.Series(df2.columns).str.endswith('time_on_oject_aggregated')])
list_col2 = list(df2.columns[pd.Series(df2.columns).str.endswith('id')])

list_col1.extend(list_col2)

In [24]:
df_out3 = pd.merge(df_out2, df2[list_col1],left_on=['session_id'],right_on=['id'], how='left')

In [25]:
del df_out3['id_x']
del df_out3['id_y']

### Drop sessions that have a checkout in first five rows

In [26]:
df_out4 = df_out3.drop(df_out3[(df_out3['no_checkout_infirstrows'] == False)].index)

### Write file and save it to working directory

In [29]:
start2=datetime.now()

df_out4.to_hdf('DataFrame_1_five_pages_df_out.h5', 'df_out', mode='w')

end2 = datetime.now()
print(end2 - start2)

C:\Users\Sellit\Anaconda3\lib\site-packages\pandas\core\generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block5_values] [items->['session_id', 'page_cat', 'uri', 'websitePage', 'ref_gen', 'page_id', 'tags1', 'title', 'country', 'region', 'city', 'isp', 'ip', 'lang', 'userAgent', 'browser', 'browserVersion', 'os', 'osVersion', 'device', 'referrer', 'referrerType', 'first_5_pages', 'screenRes', 'tags2', 'variables', 'lng', 'lat', 'annotations']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


0:00:00.817091
